<a href="https://colab.research.google.com/github/Jofdiazdi/TalleresSimpy/blob/master/Talleres/1.%20Introduccion%20a%20simpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial Simpy








 ## Conceptos basicos

Simpy es una libreria en python de simulacion discreta en donde el comportamiento de componentes activos como vehiculos, mensajes o clientes es modelado a partir de procesos. 

* **Proceso**: Un proceso e una funcion que modela el comprotamiento de una parte del modelo a estudiar. Los procesos "viven" en un ambiente y durante su ejecuccion pueden crear eventos e interactuar con otros procesos.

* **Ambiente**: Un ambiente es un entorno el cual nos proporciona la creacion y control de un reloj de simulacion y la capacidad de crear procesos y asociarlos al ambiente.

* **Evento**: Son sucesos instantaneos, es decir duracion 0 los cuales permiten la interaccion entre los procesos del ambiente. Un proceso puede se tratado como un evento Simpy.

Primero tenemos que instalar Simpy usando este comando:


In [2]:
!pip install simpy #Instalamos simpy

Y es muy imortante importar al libreria antes de usarla:

In [0]:
import simpy #Importamos simpy

 ### Procesos y el ambiente de ejecucion
 
Miremos el siguiente ejemplo del modelo de un carro el cual pasa por 2 eventos, parquear y arrancar:

In [0]:
def carro(env):
  while True :
    print('Empezo a parquear en el tiempo %d' % env.now)
    duracion_parqueo = 5
    yield env.timeout(duracion_parqueo)
    
    print('Arranco en el tiempo %d' % env.now)
    duracion_viaje = 2
    yield env.timeout(duracion_viaje)
    
env = simpy.Environment()
env.process(carro(env))
env.run(until=15)

Empezo a parquear en el tiempo 0
Arranco en el tiempo 5
Empezo a parquear en el tiempo 7
Arranco en el tiempo 12
Empezo a parquear en el tiempo 14


La funcion `carro(evn)` modela el comportamiento del carro, por lo tanto es un proceso.

Posemos distinguir 3 momentos importantes en el:
```
print('Empezo a parquear en el tiempo %d' % env.now) #1
duracion_parqueo = 5 #2
yield env.timeout(duracion_parqueo)#3

```

* La primera linea nos define un evento, muy sencillo, el cual nos avisa que el carro empezo a parquear. La declaracion `env.now` retorna el tiempo de reloj en ese instante de la simulacion.

* La segunda linea nos define el tiempo que va a durar el proceso de parqueo o mejor dicho cuanto va a durar el carro parqueado.

* La tercera linea programa un evento que se va a disparar en el tiempo (now + delay) definido con la funcion `env.timeout(delay)`

Con respecto a lo anterior, las sigueintes 3 lineas son muy similares en estructura y nos modelan cuando el carro empieza a arrancar y cuanto dura en ese estado.

```
env = simpy.Environment()
env.process(carro(env))
env.run(until=15)

```

La funcion `Enviroment()` de simpy nos proporciona un ambiente de ejecusion donde se inicializara un reloj y se correran los procesos.

La funcion del `env.process(carro(env))` genera un proceso y lo asocia al ambiente creado.

Por ultimo la funcion `env.run(until=15)` simula el sistema por 15 unidades de tiempo

### Interaccion entre procesos

Los procesos en simpy pueden ser tratados como eventos, es decir al crear un proceso dentro de la ejecucion de otro con el comando yield, el ambiente iniciara el proceso creado y solo volvera el actual cuando alla terminado.

Miremos esto en el siguiente ejemplo:

Supongamos que ahora cuando nuestro carro parquea, tambien empieza a cargar durante un tiempo.

In [0]:
import simpy

def carga(env, duracion):
  print('Va en 25%')
  print('Va en 50%')
  print('Carga completa')
  yield env.timeout(duracion)
  
def carro(env):
  while True:
    print('Comienza a parquear y cargar en el tiempo %d' % env.now)
    
    duracion_carga = 5
    
    yield env.process(carga(env,duracion_carga))
    
    print('Arranco en el tiempo %d' % env.now)
    
    duracion_viaje = 2
    yield env.timeout(duracion_viaje)

    print('Viaje termina en el tiempo %d' % env.now)

    
env = simpy.Environment()
env.process(carro(env))
env.run(until=15)

Comienza a parquear y cargar en el tiempo 0
Va en 25%
Va en 50%
Carga completa
Arranco en el tiempo 5
Viaje termina en el tiempo 7
Comienza a parquear y cargar en el tiempo 7
Va en 25%
Va en 50%
Carga completa
Arranco en el tiempo 12
Viaje termina en el tiempo 14
Comienza a parquear y cargar en el tiempo 14
Va en 25%
Va en 50%
Carga completa


Para modelar el comportameinto descrito anteriormente, tenemos que crear otro proceso el cual se encargara de modelar el comportamiento del carro cuando se esta cargando: `carga()`

Vemos que este proceso es creado durante el proceso central del carro. Y solo vuelve a el cuando se a finalizado. Ademas carga tambien demora cierto tiempo y al finalizarse vemos como aumenta el reloj de simulacion a dicho tiempo.



#### Interrupcion de procesos


Ahora piensa que no quieres esperar a que el carro termine de cargar para manejarlo.


In [5]:
import simpy

def carga(env, duracion):
  print('Va en 25%')
  print('Va en 50%')
  yield env.timeout(duracion)
  print('Carga completa')
  
def carro(env):
  while True:
    print('Comienza a parquear y cargar en el tiempo %d' % env.now)
    
    duracion_carga = 5
    try:
      yield env.process(carga(env,duracion_carga))
    except simpy.Interrupt:
      print('Carga fue interrumpida')
    
    print('Arranco en el tiempo %d' % env.now)
    
    duracion_viaje = 2
    yield env.timeout(duracion_viaje)

    print('Viaje termina en el tiempo %d' % env.now)

def driver(env, car):
  yield env.timeout(3)
  car.interrupt()
    
env = simpy.Environment()
car = env.process(carro(env))
driver = env.process(driver(env,car))
env.run(until=15)

Comienza a parquear y cargar en el tiempo 0
Va en 25%
Va en 50%
Carga fue interrumpida
Arranco en el tiempo 3
Carga completa
Viaje termina en el tiempo 5
Comienza a parquear y cargar en el tiempo 5
Va en 25%
Va en 50%
Carga completa
Arranco en el tiempo 10
Viaje termina en el tiempo 12
Comienza a parquear y cargar en el tiempo 12
Va en 25%
Va en 50%


Notamos que con respecto al codigo anterior no cambia mucho, vamos a resaltar un par de lineas


```
def driver(env, car):
  yield env.timeout(3)
  car.interrupt()
```
Aqui se esta creando otro proceso el cual toma como argumento el carro el cual se va a interrumpir.

Despues de esperar 3 unidades de tiempo, se interrumpe el proceso carro con la funcion `interrupt()`.



```
 try:
      yield env.process(carga(env,duracion_carga))
    except simpy.Interrupt:
      print('Carga fue interrumpida')
```

Algo muy importante es que cuando un procesos es interumpido se levanta una exepcion de interrumpcion, la cual tendremos que atrapar para que el programa no se caiga. En este caso, mientras esperamos a que el carro termine de cargarse, tambien preveemos que el conductor interrumpa el proceso.

Mas informacion sobre el uso de try...except puede ser consultada aqui: https://www.w3schools.com/python/python_try_except.asp





## Referencias

Ejemplos y definiciones tomadas de la pagina oficial de simpy: https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html